In [ ]:
from pydantic import BaseModel
from ogpu.client import publish_task, TaskInfo, TaskInput, ChainConfig, ChainId
from web3 import Web3

ChainConfig.set_chain(chain_id=ChainId.OGPU_MAINNET)

In [ ]:
class Inference(BaseModel):
    input: str | dict
    args : dict  = {}

In [ ]:
from typing import List, Dict, Any
from pydantic import BaseModel

class DetectionResponse(BaseModel):
    results: List[Dict[str, Any]]
    
    def display_results(self):
        """Display detection results in a nice format"""
        print("🔍 Object Detection Results")
        print("=" * 50)
        
        # Sort by confidence score (highest first)
        sorted_results = sorted(self.results, key=lambda x: x['score'], reverse=True)
        
        for i, result in enumerate(sorted_results, 1):
            confidence_percent = result['score'] * 100
            
            # Color coding based on confidence
            if confidence_percent >= 90:
                confidence_emoji = "🟢"
            elif confidence_percent >= 70:
                confidence_emoji = "🟡"
            else:
                confidence_emoji = "🔴"
                
            print(f"\n{i}. {confidence_emoji} {result['label'].upper()}")
            print(f"   Confidence: {confidence_percent:.1f}%")
            box = result['box']
            print(f"   Bounding Box: ({box['xmin']}, {box['ymin']}) → ({box['xmax']}, {box['ymax']})")

In [ ]:
inference_data = Inference(
    input= "https://lh7-us.googleusercontent.com/GNVsVvVmCDLtFtqburTWaTQOjXn3N0wLBKXt6BhXO2GG037S9o7xZ7HyPjFcJoWgds1PFKvbHgCCEWTqiB-MR0VFIo7UC7jVEX6t10LD6zPg61YoEAR-NGbA2RJj7dVdYwMH2nsOJY648uXmMlpNiZw",
    args= {
        "threshold": 0.5  # minimum confidence score
    }
)

task_input = TaskInput(
    function_name="inference",
    data = inference_data
)

In [ ]:
task_info = TaskInfo(
    source="0xaddy",
    config=task_input,
    expiryTime=86400,  # 24 hour in seconds
    payment=Web3.to_wei(0.001, "ether"),
)

task_address = publish_task(task_info)
print(f"Task published at address: {task_address}")

In [ ]:
from ogpu.client import get_task_responses
responses = get_task_responses(task_address)


In [ ]:
# detection_response = DetectionResponse(results=output)
# detection_response.display_results()